In [ ]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs
import dsds.transform as t

In [ ]:
df = pl.read_csv("../data/train.csv")
print(df.shape)
df.head(2)

In [ ]:
from dsds._rust import rs_tfidf_vectorizer, rs_ref_table

res = rs_tfidf_vectorizer(df, "Description", "snowball", 0.02, 0.95, 5000, 500, True)
res.head()

In [ ]:
%%timeit
res = rs_tfidf_vectorizer(df, "Description", "snowball", 0.02, 0.95, 5000, 500, True)

In [ ]:
c = "Description"
df = df.with_columns(
    pl.col(c).str.to_lowercase()
)

In [ ]:
rs_ref_table(df, "Description", "snowball", 0.02, 0.95, 5000, 500).sort("ref")

In [ ]:
import dsds.text as tt
res = tt.tfidf_vectorizer(df, "Description", "snowball", 0.02, 0.95, 5000, 500, True)
res.head()

In [ ]:
%%timeit
res = tt.tfidf_vectorizer(df, "Description", "snowball", 0.02, 0.95, 5000, 500, True)

In [ ]:
from dsds._rust import rs_cnt_vectorizer, rs_ref_table, rs_snowball_stem, rs_levenshtein_dist
import dsds.text as tt
c = "Ad Topic Line"

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
corpus = df["Description"].to_list()

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.02)
X = vectorizer.fit_transform(corpus).toarray()
cols = vectorizer.get_feature_names_out().tolist()
df_words = pl.from_numpy(X, schema=cols)
df_combined = pl.concat([df, df_words], how="horizontal")
df_combined

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
corpus = ['this is the first document', 'this document is the second document', 'and this is the third one','is this the first document']
vocabulary = ['this', 'document', 'first', 'is', 'second', 'the','and', 'one']
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)), ('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()
pipe['tfid'].idf_
pipe.transform(corpus).shape

In [ ]:
pipe.transform(corpus).toarray() # .shape

In [ ]:
pipe["count"].get_feature_names_out().tolist()

In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
import dsds.metrics as me
from sklearn.metrics import log_loss

def pure_numpy_logloss(y_actual:np.ndarray, y_predict:np.ndarray):
    return -np.mean(y_actual * np.log(y_predict) + (1 - y_actual) * np.log(1 - y_predict))

pred = np.random.random(size=500_000) # Some random fake predictions
actual = np.round(np.random.random(size=500_000)).astype(np.int8) # Some random fake actual labels
# Yielding the same result up to 12 digits
print(round(me.logloss(actual, pred), 12) == round(log_loss(actual, pred), 12))
print(round(me.logloss(actual, pred), 12) == round(pure_numpy_logloss(actual, pred), 12))

In [ ]:
%%timeit
me.logloss(actual, pred) # dsds

In [ ]:
%%timeit
log_loss(actual, pred) # sklearn

In [ ]:
%%timeit
pure_numpy_logloss(actual, pred) # pure numpy

In [ ]:
# from sklearn.datasets import make_classification
# orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# # This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# # You must turn other dataframe formats into Polars for dsds to work.
# df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# # Turn it into Pandas.
# df_pd = df.to_pandas()
# target = "target"
# features = df.columns
# features.remove(target)

In [ ]:
df = pl.DataFrame({
    "a": [1, np.nan, None],
    "b": [1,2,3]
})
df

In [ ]:
ps.invalid_inferral(df, threshold=0.5, include_null=True)

In [ ]:
ps.discrete_inferral(df)

In [ ]:
temp = ps.get_unique_count(df.with_row_count(offset=1).set_sorted("row_nr"), include_null_count=True)
len_df = temp.filter(pl.col("column") == "row_nr").item(0,1)
print(len_df)
temp

In [ ]:
from sklearn.datasets import make_classification
orig_x, orig_y = make_classification(n_samples = 50_000, n_features = 50, n_informative = 20, n_redundant = 30)
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
features = df_pl.columns
features.remove("target")

In [ ]:
for train, test in sa.time_series_split(df_pl, n_splits=5, offset = 1000):
    print(train.shape)
    print(test.shape)

In [ ]:
%%timeit
fs.discrete_ig(df_pl, target="target", cols=features)

In [ ]:
%%timeit
fs.discrete_ig2(df_pl, target="target", cols=features)

In [ ]:

me.logloss(actual, pred, check_binary=False)

In [ ]:
import nltk
from nltk.corpus import stopwords